In [0]:
import numpy as np
import torch
import torch.nn as nn

# 1. Introduction

### 1.1 Random matrix

In [2]:
torch.rand(5, 3)

tensor([[0.0750, 0.8932, 0.8225],
        [0.2799, 0.9963, 0.1660],
        [0.9836, 0.5259, 0.4346],
        [0.1090, 0.9594, 0.1183],
        [0.7944, 0.4027, 0.4745]])

### 1.2 Zero matrix

In [3]:
torch.zeros(5, 3, dtype=torch.long)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

### 1.3 Tensor

In [4]:
torch.tensor([5.5, 3])

tensor([5.5000, 3.0000])

### 1.4 Overwrite

In [5]:
x = torch.tensor([5.5, 3])

x.new_ones(5, 3, dtype=torch.double)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

In [6]:
torch.randn_like(x, dtype=torch.float)

tensor([-1.2281, -0.5443])

### 1.5 Add

In [7]:
x = torch.rand(2, 4)
y = torch.rand(2, 4)

x + y

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

In [8]:
x.add(y)

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

In [9]:
torch.add(x, y)

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

In [10]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)

result

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

In [11]:
# add in place

x.add_(y)

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

In [12]:
x

tensor([[1.3276, 0.7671, 1.1726, 1.0645],
        [1.1546, 1.1523, 0.4990, 1.0558]])

### 1.6 Resize tensors

In [13]:
x = torch.randn(4, 4)
y = x.view(16) # 1D arr
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions

print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


### 1.7 Tensor to numpy

In [14]:
a = torch.ones(5)
b = a.numpy()

print(a)
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [15]:
a.add_(123)

print(a)
print(b)

tensor([124., 124., 124., 124., 124.])
[124. 124. 124. 124. 124.]


### 1.8 Numpy to tensor

In [16]:
a = np.ones(5)
b = torch.from_numpy(a)

np.add(a, 1, out=a)
a += 1

print(a)
print(b)

[3. 3. 3. 3. 3.]
tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


### 1.9 CUDA Tensor

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[ 0.8563, -0.3412, -0.2331, -0.3536],
        [ 1.3282,  0.6738, -0.5558,  0.1061],
        [ 2.9679,  1.4826,  2.3489, -0.3947],
        [ 0.5042, -0.0970,  0.8650, -0.5044]], device='cuda:0')
tensor([[ 0.8563, -0.3412, -0.2331, -0.3536],
        [ 1.3282,  0.6738, -0.5558,  0.1061],
        [ 2.9679,  1.4826,  2.3489, -0.3947],
        [ 0.5042, -0.0970,  0.8650, -0.5044]], dtype=torch.float64)


# 2. AUTOGRAD: AUTOMATIC DIFFERENTIATION

In [18]:
x = torch.ones(2, 3, requires_grad=True)
print(x)

y = x + 2
print(y)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)


In [19]:
y.mean()

tensor(3., grad_fn=<MeanBackward0>)

In [20]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


In [21]:
print(b, b.grad_fn)
b.backward()
print(b, b.grad_fn)

tensor(524.1561, grad_fn=<SumBackward0>) <SumBackward0 object at 0x7f9f10896550>
tensor(524.1561, grad_fn=<SumBackward0>) <SumBackward0 object at 0x7f9f10896550>


In [22]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-356.0541, -326.8693, -992.3766], grad_fn=<MulBackward0>)


In [23]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


# 3. Neural net

In [24]:
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operations: y = Wx + b
        self.fc1 = nn.Linear(32 * 32, 200)
        self.fc2 = nn.Linear(200, 10)
        
    def forward(self, x):
        # x.shape => BATCH_SIZE x Height X Width 
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x 
    

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)


In [25]:
class Net_with_sequential(nn.Module):

    def __init__(self):
        super(Net_with_sequential, self).__init__()
        # an affine operations: y = Wx + b
        self.dense_layers = nn.Sequential(
            nn.Linear(32 * 32, 200),
            nn.ReLU(),
            nn.Linear(200, 10),
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dense_layers(x)
        return x 
    

net_ws = Net_with_sequential()
on_GPU=net_ws.to(device)
print(net_ws)

Net_with_sequential(
  (dense_layers): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=10, bias=True)
  )
)


In [26]:
IN = torch.rand(1, 1, 32, 32)
out = net(IN)

print(out)

tensor([[ 0.3463, -0.2621, -0.0353, -0.1105, -0.1398,  0.1535, -0.0291, -0.1528,
         -0.1935,  0.2155]], grad_fn=<AddmmBackward>)


In [27]:
#  Zero the gradient buffers of all parameters and backprops with random gradients:

net.zero_grad()
out.backward(torch.randn(1, 10))

print(out)

tensor([[ 0.3463, -0.2621, -0.0353, -0.1105, -0.1398,  0.1535, -0.0291, -0.1528,
         -0.1935,  0.2155]], grad_fn=<AddmmBackward>)


# 4. Loss Function

In [28]:
output = net(IN)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5335, grad_fn=<MseLossBackward>)


In [29]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('fc1.bias.grad before backward')
print(net.fc1.bias.grad)

loss.backward()

print('fc1.bias.grad after backward')
print(net.fc1.bias.grad)

fc1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
fc1.bias.grad after backward
tensor([ 0.0020,  0.0000,  0.0078,  0.0000,  0.0000,  0.0000,  0.017

# 5. Update the weights

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(IN)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

### 5.1 Dataset

In [31]:
PICTURE_SIZE = 10

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, size):
        self.data = torch.zeros(size, PICTURE_SIZE, PICTURE_SIZE)
        self.data[torch.rand_like(self.data) > 0.5] = 1.
        
        self.labels = torch.zeros(size)
        self.labels[self.data.mean(dim=[1,2]) > 0.5] = 1.
    
    def __getitem__(self, item):
        # Accepts scalars, tuples and dictionaries
        return {
                'image': self.data[item],
                'label': self.labels[item]
        }
    def __len__(self):
        return self.data.shape[0]

dataset = SimpleDataset(2**13)
# __len__
print('dataset_len:', len(dataset))

# __getitem__
print(dataset[15])

for x in dataset:
    pass
print('looping sucesfull')

dataset_len: 8192
{'image': tensor([[0., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 1., 1., 1., 1., 0., 0.],
        [0., 1., 0., 0., 1., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 1., 0., 0., 1., 1., 0.],
        [1., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 1., 0., 1., 1., 1.]]), 'label': tensor(1.)}
looping sucesfull


In [0]:
BATCH_SIZE = 32

dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=1
)

In [0]:
HIDDEN_SIZE = 800

class CustomDenseNetwork(nn.Module):
    def __init__(self):
        super(CustomDenseNetwork, self).__init__()
        self.hidden = nn.Sequential(
                nn.Linear(PICTURE_SIZE * PICTURE_SIZE, HIDDEN_SIZE),
                nn.ReLU(),
                nn.Linear(HIDDEN_SIZE, 1)
        )
    def forward(self, x):
        x = x.view(-1, PICTURE_SIZE * PICTURE_SIZE)
        x = self.hidden(x)
        return x

In [34]:
EPOCHS = 30

# Move model weigths to device
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
net = CustomDenseNetwork().to(device)

optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

for e in range(EPOCHS):
    loss_avg = 0.
    acc_avg = 0.
    for i, batch_data in enumerate(dataloader):
        # Start with zeroing .grad fields in model
        net.zero_grad()
        
        # Move data to device
        images = batch_data['image'].to(device)
        labels = batch_data['label'].to(device)
        
        #Run neural net
        out = net(images)

        #Compute loss and apply autograd for model parameters
        loss = ((out.view(-1) - labels)** 2).mean()
        loss.backward()
        
        #Update network weigths
        optimizer.step()

        # Statistics generation
        res = torch.where(
                out.view(-1) > 0.5,
                torch.tensor(1., device=device),
                torch.tensor(0., device=device)
        )
        acc = (res == labels).sum()
        loss_avg += loss
        acc_avg += acc
        if i % 30 == 0 and i != 0:
            print(f'epoch:{e + 1:2d} step: {(i):3d} loss: {loss_avg / 30:.3f} acc: {acc_avg / (BATCH_SIZE * 30):.3f}')
            acc_avg = 0.
            loss_avg = 0.
        # End of statistics generation

epoch: 1 step:  30 loss: 0.250 acc: 0.606
epoch: 1 step:  60 loss: 0.232 acc: 0.608
epoch: 1 step:  90 loss: 0.227 acc: 0.647
epoch: 1 step: 120 loss: 0.217 acc: 0.659
epoch: 1 step: 150 loss: 0.216 acc: 0.667
epoch: 1 step: 180 loss: 0.211 acc: 0.684
epoch: 1 step: 210 loss: 0.207 acc: 0.713
epoch: 1 step: 240 loss: 0.195 acc: 0.745
epoch: 2 step:  30 loss: 0.199 acc: 0.788
epoch: 2 step:  60 loss: 0.186 acc: 0.784
epoch: 2 step:  90 loss: 0.192 acc: 0.767
epoch: 2 step: 120 loss: 0.186 acc: 0.780
epoch: 2 step: 150 loss: 0.183 acc: 0.791
epoch: 2 step: 180 loss: 0.185 acc: 0.783
epoch: 2 step: 210 loss: 0.173 acc: 0.816
epoch: 2 step: 240 loss: 0.172 acc: 0.833
epoch: 3 step:  30 loss: 0.172 acc: 0.856
epoch: 3 step:  60 loss: 0.168 acc: 0.834
epoch: 3 step:  90 loss: 0.165 acc: 0.831
epoch: 3 step: 120 loss: 0.158 acc: 0.859
epoch: 3 step: 150 loss: 0.156 acc: 0.846
epoch: 3 step: 180 loss: 0.158 acc: 0.853
epoch: 3 step: 210 loss: 0.156 acc: 0.854
epoch: 3 step: 240 loss: 0.156 acc

# 6.ProTip: mnist InMemDataLoder

In [0]:
class InMemDataLoader(object):
    __initialized = False
    def __init__(self, tensors, batch_size=1, shuffle=False, sampler=None,
                 batch_sampler=None, drop_last=False):
        """A torch dataloader that fetches data from memory."""
        tensors = [torch.tensor(tensor) for tensor in tensors]
        dataset = torch.utils.data.TensorDataset(*tensors)
        self.dataset = dataset
        self.batch_size = batch_size
        self.drop_last = drop_last
        
        if batch_sampler is not None:
            if batch_size > 1 or shuffle or sampler is not None or drop_last:
                raise ValueError('batch_sampler option is mutually exclusive '
                                 'with batch_size, shuffle, sampler, and '
                                 'drop_last')
            self.batch_size = None
            self.drop_last = None

        if sampler is not None and shuffle:
            raise ValueError('sampler option is mutually exclusive with '
                             'shuffle')
            
        if batch_sampler is None:
            if sampler is None:
                if shuffle:
                    sampler = torch.utils.data.RandomSampler(dataset)
                else:
                    sampler = torch.utils.data.SequentialSampler(dataset)
            batch_sampler = torch.utils.data.BatchSampler(sampler, batch_size, drop_last)

        self.sampler = sampler
        self.batch_sampler = batch_sampler
        self.__initialized = True
    
    def __setattr__(self, attr, val):
        if self.__initialized and attr in ('batch_size', 'sampler', 'drop_last'):
            raise ValueError('{} attribute should not be set after {} is '
                             'initialized'.format(attr, self.__class__.__name__))

        super(InMemDataLoader, self).__setattr__(attr, val)

    def __iter__(self):
        for batch_indices in self.batch_sampler:
            yield self.dataset[batch_indices]

    def __len__(self):
        return len(self.batch_sampler)
    
    def to(self, device):
        self.dataset.tensors = tuple(t.to(device) for t in self.dataset.tensors)
        return self

In [36]:
##### HOW TO USE! ###
# Download raw data
# !command runs shell command in jupyter notebook
!pip install -q gdown httpimport
![ -e mnist.npz ] || gdown 'https://drive.google.com/uc?id=1QPaC3IKB_5tX6yIZgRgkpcqFrfVqPTXU' -O mnist.npz

#Extracting data
with np.load('mnist.npz') as data:
    mnist_full_train_data = data['train_data'].astype('float32') / 255.0


#Fast loader initialziation
train_loader = InMemDataLoader(
    (mnist_full_train_data, ), batch_size=25, shuffle=True)
next(iter(train_loader))

Downloading...
From: https://drive.google.com/uc?id=1QPaC3IKB_5tX6yIZgRgkpcqFrfVqPTXU
To: /content/mnist.npz
55.4MB [00:00, 97.5MB/s]


(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 